## Notebook para criação da base com os valores mínimos, máximos e média por:
- SKU;
- Estado (Regional);
- Lojista;

#### A ideia é dar uma visão para o lojista dos preços praticados nacionalmente e por região (estado) 

## Iniciando o Notebook
#### Inicialização da sessão do notebook importando a biblioteca do projeto

In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='de02faf2-fbe4-466f-b8cd-44296f096a9d', project_access_token='p-25957a2b4ca812c11c01435cf691ea3068358702')
pc = project.project_context

projectSPK = Project(spark.sparkContext, 'de02faf2-fbe4-466f-b8cd-44296f096a9d', 'p-25957a2b4ca812c11c01435cf691ea3068358702')
pcSPK = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210530023517-0001
KERNEL_ID = 4c430993-2f40-48d3-9cb3-d386d53c0961


**Importando algumas bibliotecas utilizadas no projeto**

In [32]:
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

**Incluir os parâmetros de conexão para usar os arquivos de dados com PYSPARK**

In [2]:
import ibmos2spark, os
from pyspark.sql import functions as func

# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_f35d990ae08e4675b679f0f50769655c = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_f35d990ae08e4675b679f0f50769655c = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

credentials = {
    'endpoint': endpoint_f35d990ae08e4675b679f0f50769655c,
    'service_id': 'iam-ServiceId-f8ddc357-a261-42a2-a6f2-44ac38d6cc50',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': '5y97tXSIMk-o11IIlG-qhORjxg4zQ6yFprxqB54ntyX9'
}

configuration_name = 'os_f35d990ae08e4675b679f0f50769655c_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
dfSKULogista = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('skulojista.csv', 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc'))
#dfSKULogista.take(5)

dfLojista = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('lojista.csv', 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc'))
#dfLojista.take(5)

**Cria as tabelas temporárias para utilização com comandos SQL no spark**

In [3]:
dfSKULogista.createOrReplaceTempView("SKULojista")
dfLojista.createOrReplaceTempView("Lojista")

## Análise exporatória dos dados

**Analise da tabela SKULOJISTA**
Analise do campo flagativa para identificar se este campo indica se o produto está ativo ou não
Estou usando o campo idskureferencia que é o SKU da VIA, pegando somente cadastros que estejam com este campo preenchido 

In [4]:
#identificando o domínio do campo flagativa -> só tem 0 e 1 (entede-se que 0 é ativa e 1 não ativa)
spark.sql("SELECT distinct flagativa\
             FROM SKULojista \
             where idskureferencia is not null\
             ;").show()

+---------+
|flagativa|
+---------+
|        0|
|        1|
+---------+



**Identificando o domínio dos campos flagativa e flagskusaldodisponivel**
Para ver a relação entre os dois e ver se o campo flagativa quando for 0 significa que é ativo mesmo

In [5]:
spark.sql("SELECT distinct flagativa, flagskusaldodisponivel\
             FROM SKULojista \
             where idskureferencia is not null\
             ;").show()

+---------+----------------------+
|flagativa|flagskusaldodisponivel|
+---------+----------------------+
|        1|                     1|
|        0|                     1|
|        0|                     0|
|        1|                     0|
+---------+----------------------+



**diante das análises acima foi possível observar que o campo flagativa e flagskusaldodisponivel não dá para ser usado 
Diante disto vamos analisar apenas um lojista com produtos que possui precovenda > 0**

In [43]:
spark.sql("SELECT idlojista, idsku, idskureferencia, precoanterior, precovenda\
             FROM SKULojista \
             where idlojista = 14738\
             and idskureferencia is not null\
             and precovenda > 0\
             ;").show()

+---------+----------+---------------+-------------+----------+
|idlojista|     idsku|idskureferencia|precoanterior|precovenda|
+---------+----------+---------------+-------------+----------+
|    14738|      2928|        3168410|      2099.00|   3010.90|
|    14738|    273538|        3243416|       166.17|    137.90|
|    14738|   1766101|        3309575|       419.90|    450.90|
|    14738|   3986943|        4211616|      1069.00|    879.00|
|    14738|   6054828|        6054828|       143.50|    188.90|
|    14738|   7435907|        7435907|       420.90|    623.90|
|    14738|   7435929|        7435929|       157.90|    261.90|
|    14738|   9600906|       10035626|       259.90|    220.90|
|    14738|   9600943|       10035622|       259.90|    220.90|
|    14738|  11505985|       11505985|       592.30|    570.50|
|    14738|  11549428|       11607833|         0.00|    837.40|
|    14738|  11689365|       12030946|      5199.00|   7314.90|
|    14738|  11700710|       11763185|  

**Analise da tabela LOJISTA**
Identifiquei que possuimos mais de um registro para um lojista nesta tabela,ex. idlojista = 14738
identifiquei que temos o domínio 0 e 1 tb para o campo flagativa

In [6]:
spark.sql("SELECT distinct flagativa\
             FROM Lojista \
             ;").show()

+---------+
|flagativa|
+---------+
|        0|
|        1|
+---------+



**diante desta query podemos observar que o campo flagativa = 1 significa que o seller está ativo**

In [7]:
spark.sql("SELECT distinct flagativa\
             FROM Lojista \
             where idlojista = 14738\
             ;").show()

+---------+
|flagativa|
+---------+
|        1|
+---------+



In [8]:
# com essa query é possível identificar que um seller possui mais de 1 registro
spark.sql("SELECT *\
             FROM Lojista \
             where idlojista = 14738\
             ;").show()

+---------+---------+------------------+-------------+-------------------+-----------------+------+--------------------+------------+----------+--------------------+-----------+-------------------+--------------------+-----------+--------------------+-----------+----------+
|idlojista|flagativa|flaglojistadefault|classificacao|porcentagempositivo|quantidadereviews|estado|flagrestricaovendapj|retiroemloja|lojistagpa|lojistainternacional|idgrupoitem|    datadesativacao|     dataatualizacao|idcompanhia|            idseller|idsubseller|idbandeira|
+---------+---------+------------------+-------------+-------------------+-----------------+------+--------------------+------------+----------+--------------------+-----------+-------------------+--------------------+-----------+--------------------+-----------+----------+
|    14738|        1|                 0|            3|                 72|             1679|    PR|                   1|        null|      null|                null|       100

In [9]:
# obtendo apenas 1 registro
spark.sql("SELECT distinct idlojista, estado\
             FROM Lojista \
             where idlojista = 14738\
             ;").show()

+---------+------+
|idlojista|estado|
+---------+------+
|    14738|    PR|
+---------+------+



## Depois das análises, foi possível identificar que seria ncessário criar uma tabela única com preço e estado, para facilitar as próximas análises

In [4]:
#Criando uma tabela única para facitar as análises
dfSKUs = spark.sql("SELECT distinct SK.idlojista, LJ.estado, SK.idsku, SK.idskureferencia\
                      ,CAST(SK.precoanterior AS DECIMAL(15, 2)) as precoanterior, CAST(SK.precovenda AS DECIMAL(15, 2)) precovenda\
             FROM SKULojista SK, Lojista LJ\
             where SK.idskureferencia is not null\
               and SK.precovenda > 0\
               and SK.idlojista = LJ.idlojista\
               ;")

In [5]:
dfSKUs.createOrReplaceTempView("SKUS")

**Analisando os dados criados**

In [8]:
spark.sql("SELECT *\
             FROM SKUS \
             where idskureferencia = 8303336\
             ;").show()

+---------+------+-----+---------------+-------------+----------+
|idlojista|estado|idsku|idskureferencia|precoanterior|precovenda|
+---------+------+-----+---------------+-------------+----------+
|    33259|    SP| 4639|        8303336|       669.00|    699.00|
|    12793|    SP| 4639|        8303336|       669.00|    699.00|
|    31133|    SP| 4639|        8303336|       669.00|    499.00|
|    33237|    PB| 4639|        8303336|       669.00|    699.00|
|    33223|    SP| 4639|        8303336|       669.00|    699.00|
|    33261|    SP| 4639|        8303336|       669.00|    699.00|
|    33229|    PE| 4639|        8303336|       669.00|    699.00|
|    33225|    PE| 4639|        8303336|       669.00|    699.00|
|    33263|    SP| 4639|        8303336|       669.00|    699.00|
|    33233|    PE| 4639|        8303336|       669.00|    699.00|
|    33245|    BA| 4639|        8303336|       669.00|    699.00|
|    33277|    PE| 4639|        8303336|       669.00|    699.00|
|    33213

**Analisando os tipos de dados que foram criados**

In [9]:
dfSKUs.describe()

DataFrame[summary: string, idlojista: string, estado: string, idsku: string, idskureferencia: string, precoanterior: string, precovenda: string]

## Obtendo os valores máximo, mínimo e média baseado na tabela criada, por estado e SKU pois a idéia inicial é analisar o preço do catálogo do seller por região (estado)

In [10]:
spark.sql("SELECT idskureferencia \
             ,estado\
             ,min(precovenda) as minimo\
             ,max(precovenda) as maximo\
             ,min(precovenda) as media\
             FROM SKUS \
             group by idskureferencia, estado\
             order by maximo desc\
             ;").show()

+---------------+------+--------+---------+--------+
|idskureferencia|estado|  minimo|   maximo|   media|
+---------------+------+--------+---------+--------+
|        3353401|    SP|  589.00|249480.23|  589.00|
|     1503120785|    SP|99999.00| 99999.00|99999.00|
|     1503664197|    PR| 2890.00| 94133.37| 2890.00|
|       11630126|    SP|  178.90| 90000.00|  178.90|
|     1500015908|    ES|89399.90| 89399.90|89399.90|
|     1508863399|    PR|86699.90| 86699.90|86699.90|
|     1500084539|    SP|86654.09| 86654.09|86654.09|
|       15189020|    RO|79700.00| 79700.00|79700.00|
|       12413731|    RS|76000.00| 76000.00|76000.00|
|       12718569|    SP|70581.00| 70581.00|70581.00|
|     1503019592|    SP|70559.00| 70559.00|70559.00|
|     1504864479|    SP|70559.00| 70559.00|70559.00|
|     1506324331|    SC|68333.90| 68333.90|68333.90|
|        5392997|    PR|66500.00| 66500.00|66500.00|
|     1511860920|    MG|  583.70| 60000.00|  583.70|
|     1503019628|    SP|56449.00| 56449.00|564

In [6]:
#Criando a tabela temporária no spark para exportar para arquivo
dfSKUAgrupado = spark.sql("SELECT idskureferencia \
             ,estado\
             ,min(precovenda) as minimo\
             ,max(precovenda) as maximo\
             ,min(precovenda) as media\
             FROM SKUS \
             group by idskureferencia, estado\
             order by maximo desc\
             ;")
dfSKUAgrupado.createOrReplaceTempView("SKUAgrupado")

## PREÇO VENDIDO
#### Nos próximos passos vamos obter os preços praticados para analisar a visão real do preço 
#### Na sequencia vamos juntar o preço pratica com o preço de catálogo pois poderá existir situação em que um determinado produto não teve venda, 
#### daí o preço a ser usado para cálculo do médio/mínimo e média será o do catálogo
## OBS: UTILIZAMOS 60 dias como parâmetro para buscar os preços
#### utilizamos 60 dias pois o giro de mercadoria na média é neste intervalo

In [7]:
#Obtendo dados da tabela de compras entregas por sku e criar a tabela temporária
dfCompraEntregaSKU = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compraentregasku.csv', 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc'))
dfCompraEntregaSKU.createOrReplaceTempView("compraentregasku")
dfCompraEntregaSKU.take(1)

[Row(idcompraentregasku='61380787', idcompraentregaskupai=None, iddescontocomprejunto=None, iddescontobrindecausa=None, idcompraentrega='74476290', idsku='11728496', idestoqueprovavel='7', valorvendaunidadesemdesconto='169.90', valorvendaunidade='169.90', valorcupomnominal='0.00', valorvendaunidademenoscupomnominal='169.90', valorfretecomdesconto='72.60', valorfrete='72.60', flagestoqueimpacto='1', presentemensagem=None, presentede=None, presentepara=None, idskukitsige=None, tipocombosige=None, datacriacaoregistro='2019-06-01 00:15:56.447', valorbogof='0.00', parceirorecomendacao=None, flagajusteoperacional=None, valorfreteoriginalnaorentavel='72.60', statustdca=None, valorst='0.00', sequencial='1', prazogarantiafornecedor='12', tipoestoque='N', valorjurosunitario=None, idskureferencia='11728496', precoiofvalorvenda='0.00', valorcomissao=None, percentualComissao=None, idbandeira='7')]

In [8]:
#Obtendo dados da tabela de compras entregas e criar a tabela temporária
dfCompraEntrega = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compraentrega.csv', 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc'))
dfCompraEntrega.createOrReplaceTempView("compraentrega")
dfCompraEntrega.take(1)

[Row(idcompraentrega='82908258', idcompra='166020553', idfreteentregatipo='17', idcompraentregastatus='VAL', identificadorfrete=None, idadministradorstatus='1', datasaida=None, dataprevisao=None, dataentrega=None, datastatus='2019-06-05 17:55:00', prazoentregamaisdisponibilidade=None, prazoestoque=None, gerencialid='16602055302', dataemissaonotafiscal=None, idtransportadora=None, codigocontratotransportadora=None, dataentregacorrigida=None, numctrc=None, idnotafiscal=None, idperiodoentrega=None, dataentregaagendada=None, idregiao='0', flag_chave_nfe='0', dataprometidaoriginal=None, idlojista='14406', flagenviadomarketplace='0', gerencialidmktp=None, prazotransportadora='0', prazocd='0', idfilial='1', origem='TD', ordemdevolucao='0', sequencialcompra='2', flagbloqueado=None, ordemvendaerp='B20139885', flagprazoentregamarcado=None, nomeautorizaretireloja=None, rgautorizaretireloja=None, enviadolojistagpa='0', datalimitesaidacd='2019-06-05 00:00:00', sequencialprocesso='2', dataordemvenda

In [9]:
#Obtendo dados da tabela de compras e criar a tabela temporária
dfCompra = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('compra.csv', 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc'))
dfCompra.createOrReplaceTempView("compra")
dfCompra.take(1)

[Row(idcompra='165996430', idcliente='11086617', idadministradortelevenda=None, idlistadecompra=None, midia='cpc', parceiro='rtbhouse', campanha=None, palavrachave='rtbhouse', origem=None, data='2019-06-01 10:03:00', valortotalcomdesconto='387.30', valortotalcomdesconto_devido='387.30', testeab=None, flagativa='0', statusintegracao=None, ipcliente='177.67.247.195', codigopromocao=None, infocomplcodigopromocao=None, servidororigem='CARRINHO-EX12', codloja=None, idlojafisica=None, idopcaoentregaexpressa=None, flagrespostapromocao=None, idcanalvenda='SITE', datacriacaoregistro='2019-06-04 06:02:53.520', respostapromocao=None, flag_cupom='0', idenderecolojafisica=None, useragent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36', dtajusteoperacional=None, idtipopedido='1', flaggerapontos=None, dataaceiteclubeextra=None, idtipoimposto=None, cif_fob=None, pedidoexterno=None, sequencial='1', flagcomprasige=None, flagaprovado='

## Identificar o período de dados existente na base

In [1]:
spark.sql("SELECT min(cast(CP.data as date)) \
                 ,max(cast(CP.data as date)) \
             FROM compra CP\
                 ,compraentrega CE\
                 ,compraentregasku CES\
            where CP.idcompra = CE.idcompra \
              and CE.idcompraentrega = CES.idcompraentrega \
             ;").show()

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210530160935-0000
KERNEL_ID = 41b8ea04-c4d9-408b-85ec-ecf0a2759553


AnalysisException: Table or view not found: compra; line 1 pos 97;
'Project [unresolvedalias('min(cast('CP.data as date)), None), unresolvedalias('max(cast('CP.data as date)), None)]
+- 'Filter (('CP.idcompra = 'CE.idcompra) AND ('CE.idcompraentrega = 'CES.idcompraentrega))
   +- 'Join Inner
      :- 'Join Inner
      :  :- 'SubqueryAlias CP
      :  :  +- 'UnresolvedRelation [compra]
      :  +- 'SubqueryAlias CE
      :     +- 'UnresolvedRelation [compraentrega]
      +- 'SubqueryAlias CES
         +- 'UnresolvedRelation [compraentregasku]


## Relacionando as bases temporárias para o período de COMPRAS realizadas em até 60 dias
#### Observações:
##### 1) A base de COMPRA se encontra com informações até o dia 30-04-2021 conforme visto no passo antes.
##### 2) Para uma base produtiva "online" é interessante fazer a seguinte substituição no código:
##### - DE: and cast(CP.data as date) >= cast('2021-04-30' as date) - 60
##### - PARA: and cast(CP.data as date) >= current_date() - 60
##### 3) Foi considerado para este MVP somente os seguintes sellers:
##### - idlojista in ('10012','37450','92893','80333','64189')
## Atenção: recomendado que se tire tal filtro para um ambiente produtivo deste notebook)

In [16]:
spark.sql("SELECT distinct CS.idskureferencia \
                 ,L.idlojista, L.estado \
                 ,cast(CS.valorfrete as decimal(15,2)) \
                 ,cast(CS.valorvendaunidadesemdesconto as decimal(15,2)) as valor \
             FROM compra CP\
                 ,compraentrega CE\
                 ,compraentregasku CS\
                 ,Lojista L\
            where CP.idcompra = CE.idcompra \
              and CE.idcompraentrega = CS.idcompraentrega \
              and L.idlojista = CE.idlojista\
              and cast(CP.data as date) >= cast('2021-04-30' as date) - 60 \
             ;").show()

+---------------+---------+------+----------+-------+
|idskureferencia|idlojista|estado|valorfrete|  valor|
+---------------+---------+------+----------+-------+
|       14245852|    31065|    SP|      9.38|  99.99|
|     1509924831|    11121|    RS|     59.90| 139.90|
|       15398757|    13728|    SP|     21.89| 114.90|
|       14803748|    35430|    SP|      6.38| 146.90|
|       13619054|    31534|    PR|     53.50| 239.90|
|     1510681709|    56358|    ES|      2.61|   3.32|
|     1500199983|    19937|    RS|     69.90|1069.00|
|     1508520142|    12231|    PR|     16.33|  34.99|
|     1511459552|    12231|    PR|      0.00|  89.90|
|       12735627|    10172|    SP|      5.70|   4.75|
|     1500892287|    36642|    SP|     48.58|1999.00|
|     1509342376|    32365|    ES|     54.18| 209.00|
|     1501157230|    36642|    SP|     28.23| 139.99|
|     1509515865|    44362|    MG|      7.32|  86.90|
|       11437388|    26255|    RJ|     -6.11| -39.99|
|     1502297692|    12231| 

**Juntando as tabelas de compras para obter os valores de frete e preço praticado para obter os valores finais depois**
A idéia é usar o preço do frete como indicador para o seller também

In [10]:
dtSKUSvendidos = spark.sql("SELECT distinct CS.idskureferencia \
                 ,L.idlojista, L.estado \
                 ,cast(CS.valorfrete as decimal(15,2)) \
                 ,cast(CS.valorvendaunidadesemdesconto as decimal(15,2)) as valor \
             FROM compra CP\
                 ,compraentrega CE\
                 ,compraentregasku CS\
                 ,Lojista L\
            where CP.idcompra = CE.idcompra \
              and CE.idcompraentrega = CS.idcompraentrega \
              and L.idlojista = CE.idlojista\
              and cast(CP.data as date) >= cast('2021-04-30' as date) - 60 \
             ;")
dtSKUSvendidos.createOrReplaceTempView("SKUSvendidos")

## Juntando as tabelas criadas preço de catálogo e preço praticado para ter uma única tabela 
## Atenção: Para utilização em ambiente produtivoa, precisa retirar o filtro dos sellers selecionados para o MVP

In [18]:
spark.sql("SELECT idlojista, estado, idskureferencia \
                 ,valor as precovenda, valorfrete \
             FROM SKUSvendidos \
           union all  \
           SELECT idlojista, estado, idskureferencia \
                 ,precovenda, 0 as valorfrete \
             FROM SKUS SK \
             where idlojista IN (10012,37450,92893,80333,64189) \
              and idskureferencia not in ( select idskureferencia \
             from SKUSvendidos SV where SK.idskureferencia = SV.idskureferencia) \
             ;").show()

+---------+------+---------------+----------+----------+
|idlojista|estado|idskureferencia|precovenda|valorfrete|
+---------+------+---------------+----------+----------+
|    31065|    SP|       14245852|     99.99|      9.38|
|    11121|    RS|     1509924831|    139.90|     59.90|
|    13728|    SP|       15398757|    114.90|     21.89|
|    35430|    SP|       14803748|    146.90|      6.38|
|    31534|    PR|       13619054|    239.90|     53.50|
|    56358|    ES|     1510681709|      3.32|      2.61|
|    32252|    RS|       14502946|    219.90|     49.47|
|    36913|    GO|       11704305|    139.99|      7.50|
|    19937|    RS|     1500199983|   1069.00|     69.90|
|    12231|    PR|     1508520142|     34.99|     16.33|
|    12231|    PR|     1511459552|     89.90|      0.00|
|    10172|    SP|       12735627|      4.75|      5.70|
|    36642|    SP|     1500892287|   1999.00|     48.58|
|    32365|    ES|     1509342376|    209.00|     54.18|
|    36642|    SP|     15011572

In [13]:
#criando uma tabela temporária para gerar os valores depois
dfvaloresSKUs = spark.sql("SELECT idlojista, estado, idskureferencia \
                 ,valor as precovenda, valorfrete \
             FROM SKUSvendidos \
           union all  \
           SELECT idlojista, estado, idskureferencia \
                 ,precovenda, 0 as valorfrete \
             FROM SKUS SK \
             where idlojista IN (10012,37450,92893,80333,64189) \
              and idskureferencia not in ( select idskureferencia \
             from SKUSvendidos SV where SK.idskureferencia = SV.idskureferencia) \
             ;")
dfvaloresSKUs.createOrReplaceTempView("valoresSKUs")

## Obter os valores médio, mínimo e máximo

In [20]:
spark.sql("SELECT estado, idskureferencia \
                 ,min(ABS(precovenda)) as minVrVenda \
                 ,max(ABS(precovenda)) as maxVrVenda \
                 ,mean(ABS(precovenda)) as mediaVrVenda \
                 ,min(ABS(valorfrete)) as minVrFrete \
                 ,max(ABS(valorfrete)) as maxVrFrete \
                 ,mean(ABS(valorfrete)) as mediaVrFrete\
             FROM valoresSKUs \
             group by estado, idskureferencia \
             ;").show()

+------+---------------+----------+----------+------------+----------+----------+------------+
|estado|idskureferencia|minVrVenda|maxVrVenda|mediaVrVenda|minVrFrete|maxVrFrete|mediaVrFrete|
+------+---------------+----------+----------+------------+----------+----------+------------+
|    PR|       13618740|    239.90|    239.90|  239.900000|     23.10|    157.13|   64.160465|
|    SP|        7496708|    184.90|    184.90|  184.900000|     61.42|     61.42|   61.420000|
|    SP|     1509840277|   1549.00|   1549.00| 1549.000000|     30.62|     63.36|   46.990000|
|    PR|     1504162920|   1899.99|   1899.99| 1899.990000|     22.00|     84.31|   48.560000|
|    SC|        6804778|     35.88|     35.88|   35.880000|     16.09|     16.09|   16.090000|
|    SP|     1505205640|    209.00|    209.00|  209.000000|     22.52|     22.52|   22.520000|
|    ES|     1506693425|    428.00|    449.00|  442.100000|     39.41|     87.71|   52.944000|
|    PR|     1510298611|     72.88|     74.90|   7

In [14]:
# criando um dataframe da tabela gerada para gravar em disco
dfTabelaFinal = spark.sql("SELECT estado, idskureferencia \
                 ,min(ABS(precovenda)) as minVrVenda \
                 ,max(ABS(precovenda)) as maxVrVenda \
                 ,mean(ABS(precovenda)) as mediaVrVenda \
                 ,min(ABS(valorfrete)) as minVrFrete \
                 ,max(ABS(valorfrete)) as maxVrFrete \
                 ,mean(ABS(valorfrete)) as mediaVrFrete\
             FROM valoresSKUs \
             group by estado, idskureferencia \
             ;")

## Os próxmos passos é para gravação do reultado obtido em uma tabela (arquivo) em disco

### Listando as pastas que estão na raiz do projeto

In [21]:
ls

conda/  logs/  spark-events/  user-libs/


## Criando uma pasta temporária: tmp

In [22]:
mkdir tmp

In [23]:
ls

conda/  logs/  spark-events/  tmp/  user-libs/


In [24]:
pwd

'/home/spark/shared'

In [15]:
dfTabelaFinal.repartition(1).write.mode("overwrite").parquet("/home/spark/shared/tmp/variacao_preco.parquet")

In [16]:
dfTabelaFinal.repartition(1).write.mode("overwrite").csv('/home/spark/shared/tmp/variacao_preco.csv')

In [27]:
cd /home/spark/shared/tmp

/home/spark/shared/tmp


In [29]:
ls variacao_preco.parquet/

part-00000-55badd35-9ca5-4fe7-810a-d91f70baf211-c000.snappy.parquet  _SUCCESS


In [30]:
ls variacao_preco.csv/

part-00000-ccec852c-305e-456c-a653-58d1917ae35b-c000.csv  _SUCCESS


In [33]:
#Credenciais para gravação em disco e disponibilizar nos ativos
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-f8ddc357-a261-42a2-a6f2-44ac38d6cc50',
    'IBM_API_KEY_ID': '5y97tXSIMk-o11IIlG-qhORjxg4zQ6yFprxqB54ntyX9',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc',
}

cgsClient = ibm_boto3.client(service_name='s3',
    ibm_api_key_id = credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [35]:
cgsClient.upload_file(Filename='variacao_preco.parquet/part-00000-55badd35-9ca5-4fe7-810a-d91f70baf211-c000.snappy.parquet',Bucket=credentials_1['BUCKET'],Key='variacao_preco.parquet')

In [36]:
cgsClient.upload_file(Filename='variacao_preco.csv/part-00000-ccec852c-305e-456c-a653-58d1917ae35b-c000.csv',Bucket=credentials_1['BUCKET'],Key='variacao_preco.csv')